In [19]:
import os, importlib
from codemate.embedding import code_splitter
importlib.reload(code_splitter)

<module 'code_splitter' from '/workspaces/codemate/experiments/code_splitter.py'>

Review how our test set will be split by the code splitter

In [20]:
test_snippets = []

for filename in os.listdir("test_data"):
    with open(os.path.join("test_data", filename)) as f:
        test_snippet = f.read()
    test_snippets.append({"text": test_snippet, "name": os.path.basename(filename)})

In [31]:
importlib.reload(code_splitter)

for test_snippet in test_snippets:
    splitter = code_splitter.PythonCodeSplitter(enabled_node_types=["function_definition", "decorated_definition"])
    parts = splitter.split_text(test_snippet["text"])
    print(f"{test_snippet['name']}: {len(parts)}")
    longest_part = max(parts, key=lambda x: len(x))
    print(longest_part)
    print("-" * 8)

translation_api: 2
@router.post("")
def call_translate(
    request: Request, data: schemas.TranslationInput
) -> schemas.TranslationResult:
    # Limit text length to avoid abuse
    if len(data.text) > 100:
        return {"error": "text too long"}, 400

    # Get native language
    native_language = request.state.user.native_language
    try:
        native_language_code = langcodes.find(native_language).language
    except LookupError:
        native_language_code = None

    # Translate text
    response = translation_client.translate_text(
        request={
            "parent": config.GC_TRANSLATION_LOCATION,
            "contents": [data.text],
            "mime_type": "text/plain",
            "target_language_code": "en-US",
            "source_language_code": native_language_code,
        }
    )
    translation = response.translations[0].translated_text

    # Track in Mixpanel
    mixpanel.track_in_request(
        "Translate in Video Call",
        {
            "Text": 